# COVID-19 Interactive Analysis Dashboard

In [1]:
# importing libraries

# from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual, SelectMultiple
from IPython.core.display import display, HTML

import numpy as np
from numpy import convolve

import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
# import folium
import plotly.graph_objects as go
from plotly.subplots import make_subplots
# import seaborn as sns
import ipywidgets as widgets
fig = go.FigureWidget( layout=go.Layout() )


In [2]:
# loading data right from the source:
death_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
confirmed_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
recovered_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
country_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/web-data/data/cases_country.csv')
us_confirmed_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
us_death_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')

In [3]:
# data cleaning

# renaming the df column names to lowercase
country_df.columns = map(str.lower, country_df.columns)
confirmed_df.columns = map(str.lower, confirmed_df.columns)
death_df.columns = map(str.lower, death_df.columns)
recovered_df.columns = map(str.lower, recovered_df.columns)
us_confirmed_df.columns = map(str.lower, us_confirmed_df.columns)
us_death_df.columns = map(str.lower, us_death_df.columns)

# changing province/state to state and country/region to country
confirmed_df = confirmed_df.rename(columns={'province/state': 'state', 'country/region': 'country'})
recovered_df = confirmed_df.rename(columns={'province/state': 'state', 'country/region': 'country'})
death_df = death_df.rename(columns={'province/state': 'state', 'country/region': 'country'})
country_df = country_df.rename(columns={'country_region': 'country'})
us_confirmed_df = us_confirmed_df.rename(columns={'province_state': 'state', 'admin2': 'county', 'country_region': 'country'})
us_death_df = us_death_df.rename(columns={'province_state': 'state', 'admin2': 'county', 'country_region': 'country'})
# drop rows with nulls
us_confirmed_df = us_confirmed_df.dropna(axis = 0, how ='any') 
us_death_df = us_death_df.dropna(axis = 0, how ='any') 

In [4]:
# compute daily cases
confirmed_daily_df = confirmed_df.copy(deep=True)
for col in range(6, len(confirmed_daily_df.columns)):
    confirmed_daily_df[confirmed_daily_df.columns[col]] = confirmed_df[confirmed_df.columns[col]] - confirmed_df[confirmed_df.columns[col-1]]

death_daily_df = death_df.copy(deep=True)
for col in range(6, len(death_daily_df.columns)):
    death_daily_df[death_daily_df.columns[col]] = death_df[death_df.columns[col]] - death_df[death_df.columns[col-1]]

us_confirmed_daily_df = us_confirmed_df.copy(deep=True)
for col in range(12, len(us_confirmed_daily_df.columns)):
    us_confirmed_daily_df[us_confirmed_daily_df.columns[col]] = us_confirmed_df[us_confirmed_df.columns[col]] - us_confirmed_df[us_confirmed_df.columns[col-1]]

us_death_daily_df = us_death_df.copy(deep=True)
for col in range(12, len(us_death_daily_df.columns)):
    us_death_daily_df[us_death_daily_df.columns[col]] = us_death_df[us_death_df.columns[col]] - us_death_df[us_death_df.columns[col-1]]


In [5]:
# total number of confirmed, death and recovered cases
confirmed_total = int(country_df['confirmed'].sum())
deaths_total = int(country_df['deaths'].sum())
recovered_total = int(country_df['recovered'].sum())
active_total = int(country_df['active'].sum())

In [6]:
def movingaverage (values, window):
    weights = np.repeat(1.0, window)/window
    sma = np.convolve(values, weights, 'valid')
    return sma

In [7]:
def plot_cases_global(x):
    labels = ['deaths', 'confirmed', 'deaths', 'confirmed']
    colors = ['palevioletred', 'cadetblue', 'red', 'blue']
    mode_size = [6, 6, 6, 6]
    line_size = [3, 3, 3, 3]

#     region_type = 'country'
    chartType = ['bar', 'bar', 'line', 'line']
#     df_list = [confirmed_daily_df, death_daily_df, confirmed_df, death_df]
#     df_list
    
    # Create figure with secondary y-axis
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    yMaxSecondary = max(np.sum(np.asarray(df_list[1][df_list[1][region_type] == x[0]].iloc[:,60:]),axis = 0)) * 2.0
    fig.update_yaxes(range=[3, yMaxSecondary], secondary_y=True, showgrid=False)
    
    for i, df in enumerate(df_list):
        for region in x:
            x_data = np.array(list(df.iloc[:, 60:].columns))
            y_data = np.sum(np.asarray(df[df[region_type] == region].iloc[:,60:]),axis = 0)
                
            if (chartType[i] == 'line'):
                fig.add_trace(go.Scatter(x = x_data, y = y_data, line_shape = 'linear', line=dict(color=colors[i]), 
                                         name=region + ' ' + labels[i] + ' total (' + str(max(y_data)) + ')'), 
                              secondary_y = False)
                #line_shape - linear/spline
            elif (chartType[i] == 'bar'):
                fig.add_trace(go.Bar(x = x_data, y = y_data, 
                                     name=region + ' ' + labels[i] + ' daily', 
                                     marker_color = colors[i]), secondary_y = True)
    #             fig = go.Figure(data=[go.Bar(name='xxx', x=x_data, y=y_data)])
                if (i == 1):
                    yMA = movingaverage(y_data,7)
                    fig.add_trace(go.Scatter(x = x_data[len(x_data)-len(yMA):], y = yMA, line_shape = 'linear', line=dict(color=colors[i]), 
                                     name='Moving average'), 
                          secondary_y = True)

    
    fig.update_layout(
        title="COVID 19 cases by " + region_type,
        xaxis_title='Date',
        yaxis_title='No. of Confirmed Cases',
        yaxis2_title='Daily Cases (bar charts)',
        margin=dict(l=20, r=20, t=40, b=20),
        paper_bgcolor="lightgrey",
        width = 800,
        
    );
    
    fig.update_yaxes(type="linear")
    fig.show();

In [8]:
def plot_cases_state(x):
    labels = ['deaths', 'confirmed', 'deaths', 'confirmed']
    colors = ['palevioletred', 'cadetblue', 'red', 'blue']
    mode_size = [6, 6, 6, 6]
    line_size = [3, 3, 3, 3]

#     region_type = 'country'
    chartType = ['bar', 'bar', 'line', 'line']
#     df_list = [confirmed_daily_df, death_daily_df, confirmed_df, death_df]
#     df_list
    
    # Create figure with secondary y-axis
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    yMaxSecondary = max(np.sum(np.asarray(df_list_state[1][df_list_state[1][region_type_state] == x[0]].iloc[:,60:]),axis = 0)) * 2.0
    fig.update_yaxes(range=[3, yMaxSecondary], secondary_y=True, showgrid=False)
    
    for i, df in enumerate(df_list_state):
        for region in x:
            x_data = np.array(list(df.iloc[:, 60:].columns))
            y_data = np.sum(np.asarray(df[df[region_type_state] == region].iloc[:,60:]),axis = 0)
                
            if (chartType[i] == 'line'):
                fig.add_trace(go.Scatter(x = x_data, y = y_data, line_shape = 'linear', line=dict(color=colors[i]), 
                                         name=region + ' ' + labels[i] + ' total (' + str(max(y_data)) + ')'), 
                              secondary_y = False)
                #line_shape - linear/spline
            elif (chartType[i] == 'bar'):
                fig.add_trace(go.Bar(x = x_data, y = y_data, 
                                     name=region + ' ' + labels[i] + ' daily', 
                                     marker_color = colors[i]), secondary_y = True)
    #             fig = go.Figure(data=[go.Bar(name='xxx', x=x_data, y=y_data)])
                if (i == 1):
                    yMA = movingaverage(y_data,7)
                    fig.add_trace(go.Scatter(x = x_data[len(x_data)-len(yMA):], y = yMA, line_shape = 'linear', line=dict(color=colors[i]), 
                                     name='Moving average'), 
                          secondary_y = True)

    
    fig.update_layout(
        title="COVID 19 cases by " + region_type_state,
        xaxis_title='Date',
        yaxis_title='No. of Confirmed Cases',
        yaxis2_title='Daily Cases (bar charts)',
        margin=dict(l=20, r=20, t=40, b=20),
        paper_bgcolor="lightgrey",
        width = 800,
        
    );
    
    fig.update_yaxes(type="linear")
    fig.show();

In [9]:
def plot_cases_county(x):
    labels = ['deaths', 'confirmed', 'deaths', 'confirmed']
    colors = ['palevioletred', 'cadetblue', 'red', 'blue']
    mode_size = [6, 6, 6, 6]
    line_size = [3, 3, 3, 3]

#     region_type = 'country'
    chartType = ['bar', 'bar', 'line', 'line']
#     df_list = [confirmed_daily_df, death_daily_df, confirmed_df, death_df]
#     df_list
    
    # Create figure with secondary y-axis
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    yMaxSecondary = max(np.sum(np.asarray(df_list_county[1][df_list_county[1][region_type_county] == x[0]].iloc[:,60:]),axis = 0)) * 2.0
    fig.update_yaxes(range=[3, yMaxSecondary], secondary_y=True, showgrid=False)
    
    for i, df in enumerate(df_list_county):
        for region in x:
            x_data = np.array(list(df.iloc[:, 60:].columns))
            y_data = np.sum(np.asarray(df[df[region_type_county] == region].iloc[:,60:]),axis = 0)
                
            if (chartType[i] == 'line'):
                fig.add_trace(go.Scatter(x = x_data, y = y_data, line_shape = 'linear', line=dict(color=colors[i]), 
                                         name=region + ' ' + labels[i] + ' total (' + str(max(y_data)) + ')'), 
                              secondary_y = False)
                #line_shape - linear/spline
            elif (chartType[i] == 'bar'):
                fig.add_trace(go.Bar(x = x_data, y = y_data, 
                                     name=region + ' ' + labels[i] + ' daily', 
                                     marker_color = colors[i]), secondary_y = True)
    #             fig = go.Figure(data=[go.Bar(name='xxx', x=x_data, y=y_data)])
                if (i == 1):
                    yMA = movingaverage(y_data,7)
                    fig.add_trace(go.Scatter(x = x_data[len(x_data)-len(yMA):], y = yMA, line_shape = 'linear', line=dict(color=colors[i]), 
                                     name='Moving average'), 
                          secondary_y = True)

    
    fig.update_layout(
        title="COVID 19 cases by " + region_type_county,
        xaxis_title='Date',
        yaxis_title='No. of Confirmed Cases',
        yaxis2_title='Daily Cases (bar charts)',
        margin=dict(l=20, r=20, t=40, b=20),
        paper_bgcolor="lightgrey",
        width = 800,
        
    );
    
    fig.update_yaxes(type="linear")
    fig.show();

In [10]:
# displaying the total stats

display(HTML("<div style = 'background-color: #504e4e; padding: 30px '>" +
             "<span style='color: #fff; font-size:30px;'> Confirmed: "  + str(confirmed_total) +"</span>" +
             "<span style='color: red; font-size:30px;margin-left:20px;'> Deaths: " + str(deaths_total) + "</span>"+
             "<span style='color: lightgreen; font-size:30px; margin-left:20px;'> Recovered: " + str(recovered_total) + "</span>"+
             "</div>")
       )

# Check the details of your country or the World

In [11]:
country_list = confirmed_df.country.unique().tolist()
country_list.sort()
startValue = ['US',]
region_type = 'country'
df_list = [death_daily_df, confirmed_daily_df, death_df, confirmed_df]

countrySelection = SelectMultiple(
    options = country_list, # What will be available in Selection
    value = startValue, # What will first be chosen.  Here we are chossing all
    rows=10, # How many rows will be displayed initially
    description='Countries', # Text outside of the widget
    disabled=False # If 'True', the widget will not be usable
)

interact(plot_cases_global, x=countrySelection)

ipywLayout = widgets.Layout(border='solid 2px green')
ipywLayout.display='none' # uncomment this, run cell again - then the graph/figure disappears
widgets.VBox([fig], layout=ipywLayout)

interactive(children=(SelectMultiple(description='Countries', index=(174,), options=('Afghanistan', 'Albania',…

    'data': [], 'layout': {'template': '...'}
}),), layout=Layout(border='solid …

# Details by US State

In [12]:
state_list = us_confirmed_df.state.unique().tolist()
state_list.sort()
startValue = ['Texas',]
region_type_state = 'state'
df_list_state = [us_death_daily_df, us_confirmed_daily_df, us_death_df, us_confirmed_df]

stateSelection = SelectMultiple(
    options = state_list, # What will be available in wellSelection
    value = startValue, # What will first be chosen.  Here we are chossing all
    rows=10, # How many rows will be displayed initially
    description='States', # Text outside of the widget
    disabled=False # If 'True', the widget will not be usable
)

interact(plot_cases_state, x=stateSelection)

ipywLayout = widgets.Layout(border='solid 2px green')
ipywLayout.display='none' # uncomment this, run cell again - then the graph/figure disappears
widgets.VBox([fig], layout=ipywLayout)

interactive(children=(SelectMultiple(description='States', index=(43,), options=('Alabama', 'Alaska', 'Arizona…

    'data': [], 'layout': {'template': '...'}
}),), layout=Layout(border='solid …

# Details by US County

In [13]:
county_list = us_confirmed_df.county.unique().tolist()
county_list.sort()
startValue = ['Harris',]
region_type_county = 'county'
df_list_county = [us_death_daily_df, us_confirmed_daily_df, us_death_df, us_confirmed_df]

countySelection = SelectMultiple(
    options = county_list, # What will be available in wellSelection
    value = startValue, # What will first be chosen.  Here we are chossing all
    rows=10, # How many rows will be displayed initially
    description='Counties', # Text outside of the widget
    disabled=False # If 'True', the widget will not be usable
)

interact(plot_cases_county, x=countySelection)

ipywLayout = widgets.Layout(border='solid 2px green')
ipywLayout.display='none' # uncomment this, run cell again - then the graph/figure disappears
widgets.VBox([fig], layout=ipywLayout)

interactive(children=(SelectMultiple(description='Counties', index=(725,), options=('Abbeville', 'Acadia', 'Ac…

    'data': [], 'layout': {'template': '...'}
}),), layout=Layout(border='solid …